#Ecuadorian Network
Evaluation of data from Pypsa-Earth


In [66]:
"""
Setup environment and load the base PyPSA-Earth network for a specified country.
"""

import os
import sys
import warnings
import pypsa
import warnings
import pypsa
from pathlib import Path
import os
import copy
from os.path import join

# Import all dirs
parent_dir = Path(os.getcwd()).parents[0]
sys.path.append(str(parent_dir))
from src.paths import all_dirs

dirs = all_dirs()


import logging
from pathlib import Path

LOG_FILE = join(parent_dir, "logs.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.FileHandler(LOG_FILE, encoding="utf-8")],
)


# Suppress warnings
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)
# Suppress unnecessary warnings for cleaner output
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


# Define country parameters
country_code = "EC"  # ISO 2-letter code (e.g., 'GH' for Ghana, 'CO' for Colombia)
country_name = "Ecuador"  # Country name
country_gadm = "ECU"  # ISO 3-letter GADM code

# Load the base network file path
network_dir = dirs["data/raw/networks"]
network_file = "base.nc"
network_path = os.path.join(network_dir, network_file)

# Load the PyPSA network
network = pypsa.Network(network_path)
network_original = copy.deepcopy(network)

## Dictionary with all the networks
networks_dict = {
    "network_original": network_original
}

print(f"Network loaded successfully from: {network_path}")

Network loaded successfully from: c:\Repositories\Repos\pypsa-earth-project\EcuadorElectricGrid\data\raw\networks\base.nc


The values of voltage differ from the nominal values from Ecuador

In [67]:
def snap_voltages_to_ecuador_levels(network):
    """
    Snap nominal voltages (v_nom) in a PyPSA network to Ecuador's standard levels.

    Ecuador standard transmission voltages: 500, 230, 138, 69, 48 kV.

    The function modifies the PyPSA network *in place*:
      - network.buses.v_nom
      - network.lines.v_nom

    It adds backup columns 'v_nom_raw' to preserve original values.
    Prints a short summary of all changes.

    Parameters
    ----------
    network : pypsa.Network
        The PyPSA network object to modify.

    Returns
    -------
    network : pypsa.Network
        The same network object (modified in place, returned for chaining).
    """

    import numpy as np
    import pandas as pd

    VALID_VOLTAGES_KV = np.array([500.0, 230.0, 138.0, 69.0, 48.0])

    def _nearest_valid_voltage(values):
        vals = np.asarray(values, dtype=float)
        mask = ~np.isnan(vals)
        snapped = np.full_like(vals, np.nan, dtype=float)
        diffs = np.abs(vals[mask, None] - VALID_VOLTAGES_KV[None, :])
        snapped[mask] = VALID_VOLTAGES_KV[np.argmin(diffs, axis=1)]
        return snapped

    def _apply_snap(df, label):
        if "v_nom" not in df.columns:
            print(f"[WARN] '{label}' has no 'v_nom' column. Skipping.")
            return

        if "v_nom_raw" not in df.columns:
            df["v_nom_raw"] = df["v_nom"].copy()

        before = df["v_nom"].copy()
        df["v_nom"] = _nearest_valid_voltage(df["v_nom"].values)

        changed = before != df["v_nom"]
        n_changed = int(changed.sum())

        if n_changed:
            try: 
                print(f"[{label}] Updated {n_changed} entries in v_nom:")
                summary = (
                    pd.concat([before[changed], df["v_nom"][changed]], axis=1)
                    .rename(columns={0: "old", 1: "new"})
                    .value_counts()
                    .rename("count")
                )
                for (old, new), cnt in summary.items():
                    print(f"  {old:.0f} → {new:.0f} kV : {cnt}")
            except Exception as e:
                print(f"[{label}] Updated {n_changed} entries in v_nom (summary failed: {e})")
        else:
            print(f"[{label}] No changes needed.")

    # --- Apply to main components ---
    _apply_snap(network.buses, "buses")
    _apply_snap(network.lines, "lines")

    # Optional: extend here if needed
    # if hasattr(network, "transformers"):
    #     _apply_snap(network.transformers, "transformers")

    # --- Summary ---
    print("\nFinal voltage levels:")
    print("  buses :", np.sort(network.buses.v_nom.unique()))
    print("  lines :", np.sort(network.lines.v_nom.unique()))

    return network

# Apply voltage snapping
network_snapped = snap_voltages_to_ecuador_levels(network)
networks_dict["network_snapped"] = network_snapped


[buses] Updated 7 entries in v_nom:
[buses] Updated 7 entries in v_nom (summary failed: Grouper for 'v_nom' not 1-dimensional)
[lines] Updated 5 entries in v_nom:
[lines] Updated 5 entries in v_nom (summary failed: Grouper for 'v_nom' not 1-dimensional)

Final voltage levels:
  buses : [ 48.  69. 138. 230. 500.]
  lines : [ 48.  69. 138. 230. 500.]


The new components from the Master Plan and from the Expansion plans need to be added.
Additionally, appropiate buses for the nuclear option need to be added.

In [68]:
# Copy network and merge expansion data for buses, lines, and transformers using madd

import os
import pandas as pd

# Make a safe copy of the network already loaded as `network`
network_exp = copy.deepcopy(network_snapped)

# Base dir for expansion CSVs using your dirs mapping
DATA_DIR = dirs["data/raw/networks"]

# CSV file paths
BUS_EXP_CSV = os.path.join(DATA_DIR, "EC_buses_expansion.csv")
LINE_EXP_CSV = os.path.join(DATA_DIR, "EC_lines_expansion.csv")
TRAFO_EXP_CSV = os.path.join(DATA_DIR, "EC_trafo_expansion.csv")


def _filter_to_allowed_attrs(nw, component, df):
    helper_dict = {
        "Bus": nw.buses,
        "Line": nw.lines,
        "Transformer": nw.transformers,
    }
    allowed = set(helper_dict[component].columns)
    keep_cols = [c for c in df.columns if c in allowed]
    return df[keep_cols]


def _update_existing(nw, component, df):
    table_map = {
        "Bus": "buses",
        "Line": "lines",
        "Transformer": "transformers",
    }
    table = getattr(nw, table_map[component])
    existing = df.index.intersection(table.index)
    if len(existing) > 0:
        cols = df.columns
        table.loc[existing, cols] = df.loc[existing, cols]
    return existing


def _madd_new(nw, component, df):
    if len(df) == 0:
        return
    # df must be indexed by component names and include only valid attrs
    names = df.index
    kwargs = {col: df[col] for col in df.columns}
    nw.madd(component, names, **kwargs)


# 1) Buses expansion
bus_df = pd.read_csv(BUS_EXP_CSV)
if "Name" not in bus_df.columns:
    raise ValueError("EC_buses_expansion.csv must contain a 'Name' column.")
bus_df = bus_df.set_index("Bus")
bus_df = _filter_to_allowed_attrs(network_exp, "Bus", bus_df)

existing_buses = _update_existing(network_exp, "Bus", bus_df)
to_add_buses = bus_df.drop(index=existing_buses, errors="ignore")
_madd_new(network_exp, "Bus", to_add_buses)

# 2) Lines expansion
line_df = pd.read_csv(LINE_EXP_CSV)
if "Line" not in line_df.columns:
    raise ValueError("EC_lines_expansion.csv must contain a 'Line' column.")
line_df = line_df.set_index("ID")
line_df = _filter_to_allowed_attrs(network_exp, "Line", line_df)

existing_lines = _update_existing(network_exp, "Line", line_df)
to_add_lines = line_df.drop(index=existing_lines, errors="ignore")
_madd_new(network_exp, "Line", to_add_lines)


# 3) Transformers expansion
trafo_df = pd.read_csv(TRAFO_EXP_CSV)
if "Transformer" not in trafo_df.columns:
    raise ValueError("EC_trafo_expansion.csv must contain a 'Transformer' column.")
trafo_df = trafo_df.set_index("ID")
trafo_df = _filter_to_allowed_attrs(network_exp, "Transformer", trafo_df)


existing_trafos = _update_existing(network_exp, "Transformer", trafo_df)
to_add_trafos = trafo_df.drop(index=existing_trafos, errors="ignore")
_madd_new(network_exp, "Transformer", to_add_trafos)

# Summary
print(
    f"Expansion merged: +{len(to_add_buses)} buses, "
    f"+{len(to_add_lines)} lines, "
    f"+{len(to_add_trafos)} transformers; "
    f"updated {len(existing_buses)} buses, {len(existing_lines)} lines, {len(existing_trafos)} transformers."
)
# plot_buses_and_lines_by_voltage(network_exp, save_name="ecuador_buses_expansion.png")

networks_dict["network_expanded"] = network_exp

Expansion merged: +23 buses, +16 lines, +15 transformers; updated 0 buses, 0 lines, 0 transformers.


Plot only the expansion

In [69]:
#Sanity + topology checks for expansion elements against the existing network
from src.ec_network_eval import evaluate_network

_, issues = evaluate_network(
    network_exp,
    data_dir = dirs["data/raw/networks"],
    downstream_path = (500,48))

for k,v in issues:
    if "empty" not in k:
        print(k)
        display(v)


orphan_buses


,Bus,v_nom,lon,lat
109,109,48.0,-78.7385,-2.9038
122,122,69.0,-78.7498,-1.7366
134,134,69.0,-78.7659,-3.8451
135,135,69.0,-79.9708,-3.5235
168,168,69.0,-80.5068,-0.8724
172,172,69.0,-80.1647,-0.8570
180,180,69.0,-78.7958,1.2621
187,187,69.0,-80.0950,-0.8791
193,193,69.0,-79.6392,-3.7076
203,203,69.0,-80.2042,-2.3207


bus_voltage_levels


[48, 69, 138, 230, 500]

trafo_pairs_found_network


[(48, 138), (69, 138), (69, 230), (69, 500), (138, 230), (230, 500)]

topology_summary


{'230kV': {'n_components': 63, 'sizes': [5, 4, 3, 2, 2]},
 '500kV': {'n_components': 13, 'sizes': [6, 1, 1, 1, 1]}}

The orphan buses must be cleaned. Linking them with other components may lead to incorrect strcturing of the grid


In [70]:
def clean_orphan_buses(n_exp, issues, log_csv_path="removed_orphan_buses.csv"):
    """
    Remove buses that are truly unreferenced by any component in the network.
    - Deep-copies the input network (does not modify original).
    - Uses orphan candidates from issues if present.
    - Verifies true orphans by scanning all component bus references.
    - Optionally writes removed buses to CSV (set log_csv_path=None to skip)
    Returns
    - n_exp_clean: deep-copied and cleaned network
    """
    # 1) Extract candidate orphan buses from issues (accepts multiple keys)
    orphan_df = None
    for key, data in issues:
        if key in ("orphan_buses", "orphan_buses_lines_trafos_only"):
            orphan_df = data
            break

    candidate_orphans = pd.Index([])
    if orphan_df is not None and "Bus" in orphan_df.columns:
        candidate_orphans = pd.Index(orphan_df["Bus"].tolist())

    # 2) Collect all buses referenced by any component
    def buses_referenced_by_components(n):
        refs = set()
        comps = [
            ("loads", n.loads),
            ("generators", n.generators),
            ("storage_units", n.storage_units),
            ("stores", n.stores),
            ("links", n.links),
            ("lines", n.lines),
            ("transformers", n.transformers),
            ("shunt_impedances", n.shunt_impedances),
        ]
        for _, df in comps:
            if df is None or getattr(df, "empty", True):
                continue
            # Any column that starts with 'bus' is considered a bus reference
            bus_cols = [c for c in df.columns if c.startswith("bus")]
            if not bus_cols:
                continue
            for col in bus_cols:
                # Normalize to string IDs, drop NaNs
                refs |= set(df[col].dropna().astype(str))
        return refs

    ref_buses = buses_referenced_by_components(n_exp)
    # Normalize the network bus index to string for cross-compatibility
    bus_index_str = n_exp.buses.index.astype(str)
    # Buses not referenced by any component
    true_orphans_idx = n_exp.buses.loc[~bus_index_str.isin(ref_buses)].index

    # 3) Intersect with candidates (if provided)
    if len(candidate_orphans) > 0:
        candidate_orphans = pd.Index(candidate_orphans)
        true_orphans_idx = candidate_orphans.intersection(true_orphans_idx)

    # 4) Deep-copy the network and remove the confirmed orphan buses
    n_exp_clean = copy.deepcopy(n_exp)
    for b in true_orphans_idx:
        n_exp_clean.remove("Bus", b)

    # 5) Optional: log removed buses
    if log_csv_path:
        pd.DataFrame({"removed_bus": pd.Index(true_orphans_idx)}).to_csv(
            log_csv_path, index=False
        )

    # 6) Sanity assert
    for b in true_orphans_idx:
        assert b not in n_exp_clean.buses.index, f"Bus {b} was not removed properly."
    logging.info("All orphan buses cleared")

    return n_exp_clean

In [71]:
n_exp_clean = clean_orphan_buses(network_exp, issues)



In [72]:
import math
import numpy as np
import pandas as pd


def haversine_km(lat1, lon1, lat2, lon2):
    """Great-circle distance in km between two (lat, lon) points."""
    R = 6371.0
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.asin(math.sqrt(a))


def _resolve_bus_id(bus_id, buses_index):
    """
    Normalize bus id (int or str) to the string index used in network.buses.
    """
    s = str(bus_id)
    if s in buses_index:
        return s
    raise KeyError(f"Bus {bus_id} not found in network.buses.index")


def write_csv_lines(network, bus_pair_list, out_file="line_fix.csv", id_start=1):
    """
    Create a lines CSV with rows connecting the given bus pairs.

    Output CSV columns (in order):

    Line,ID,num_parallel,v_nom,bus0,bus1,length,dc,geometry,bounds,
    carrier,type,s_max_pu,s_nom,build_year,underwater_fraction,
    x,r,g,b,s_nom_mod,s_nom_extendable,s_nom_min,s_nom_max,
    capital_cost,lifetime,terrain_factor,v_ang_min,v_ang_max,
    sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,description

    Parameters
    ----------
    network : pypsa.Network
    bus_pair_list : list of (bus0, bus1)
        Each element can be int or str, but must match network.buses.index.
    out_file : str
    id_start : int
        Starting value for the ID column (default 1).

    Returns
    -------
    df : pandas.DataFrame
        The dataframe that was written.
    """
    buses = network.buses
    lines = network.lines

    required_bus_cols = {"lon", "lat", "v_nom"}
    if not required_bus_cols.issubset(buses.columns):
        missing = required_bus_cols - set(buses.columns)
        raise ValueError(f"network.buses missing required columns: {missing}")

    # Template lines by v_nom (first line per voltage)
    templates_by_v = {}
    if "v_nom" in lines.columns and not lines.empty:
        for v in sorted(lines["v_nom"].dropna().unique()):
            same_v = lines[lines["v_nom"] == v]
            if not same_v.empty:
                templates_by_v[v] = same_v.iloc[0]
    global_template = lines.iloc[0] if not lines.empty else None

    cols = [
        "Line","ID","num_parallel","v_nom","bus0","bus1","length","dc","geometry","bounds",
        "carrier","type","s_max_pu","s_nom","build_year","underwater_fraction",
        "x","r","g","b","s_nom_mod","s_nom_extendable","s_nom_min","s_nom_max",
        "capital_cost","lifetime","terrain_factor","v_ang_min","v_ang_max",
        "sub_network","x_pu","r_pu","g_pu","b_pu","x_pu_eff","r_pu_eff","s_nom_opt","description",
    ]

    rows = []

    for idx, (bus0, bus1) in enumerate(bus_pair_list):
        # --- resolve buses ---
        b0_id = _resolve_bus_id(bus0, buses.index)
        b1_id = _resolve_bus_id(bus1, buses.index)

        b0 = buses.loc[b0_id]
        b1 = buses.loc[b1_id]

        lon0, lat0 = float(b0["lon"]), float(b0["lat"])
        lon1, lat1 = float(b1["lon"]), float(b1["lat"])

        v0 = float(b0["v_nom"])
        v1 = float(b1["v_nom"])
        if not np.isclose(v0, v1):
            raise ValueError(
                f"Bus {b0_id} has v_nom={v0}, bus {b1_id} has v_nom={v1}; "
                "this pair should be a transformer, not a line."
            )
        v_nom = v0

        length_km = haversine_km(lat0, lon0, lat1, lon1)

        # MULTILINESTRING with one segment (structure like your example)
        geometry = f"MULTILINESTRING (({lon0} {lat0}, {lon1} {lat1}))"
        bounds   = ""  # example CSV had empty bounds field

        # pick template for electrical params
        template = templates_by_v.get(v_nom, global_template)

        # try to keep bus0/bus1 numeric if possible (cosmetic)
        def maybe_int_str(x):
            s = str(x)
            return int(s) if s.isdigit() else s

        bus0_out = maybe_int_str(b0_id)
        bus1_out = maybe_int_str(b1_id)

        # defaults
        row = {
            "Line":            f"fix_{b0_id}_{b1_id}",
            "ID":              id_start + idx,
            "num_parallel":    1.0,
            "v_nom":           v_nom,
            "bus0":            bus0_out,
            "bus1":            bus1_out,
            "length":          length_km,
            "dc":              False,
            "geometry":        geometry,
            "bounds":          bounds,
            "carrier":         "AC",
            "type":            "",
            "s_max_pu":        1.0,
            "s_nom":           0.0,
            "build_year":      2000,          # or 0 if you prefer
            "underwater_fraction": 0.0,
            "x":               0.0,
            "r":               0.0,
            "g":               0.0,
            "b":               0.0,
            "s_nom_mod":       0.0,
            "s_nom_extendable": False,
            "s_nom_min":       0.0,
            "s_nom_max":       float("inf"),
            "capital_cost":    0.0,
            "lifetime":        float("inf"),
            "terrain_factor":  1.0,
            "v_ang_min":       float("-inf"),
            "v_ang_max":       float("inf"),
            "sub_network":     "",
            "x_pu":            0.0,
            "r_pu":            0.0,
            "g_pu":            0.0,
            "b_pu":            0.0,
            "x_pu_eff":        0.0,
            "r_pu_eff":        0.0,
            "s_nom_opt":       0.0,
            "description":     f"{int(v_nom)} kV new line: {b0_id}_to_{b1_id}",
        }

        # overwrite from template where available
        if template is not None:
            for k in [
                "num_parallel","carrier","type","s_max_pu","s_nom",
                "build_year","underwater_fraction","x","r","g","b",
                "s_nom_mod","s_nom_extendable","s_nom_min","s_nom_max",
                "capital_cost","lifetime","terrain_factor",
                "v_ang_min","v_ang_max","sub_network",
                "x_pu","r_pu","g_pu","b_pu","x_pu_eff","r_pu_eff",
                "s_nom_opt",
            ]:
                if k in template.index:
                    row[k] = template[k]

        rows.append(row)

    df = pd.DataFrame(rows, columns=cols)
    df.to_csv(out_file, index=False)
    print(f"Wrote {len(df)} lines to {out_file}")
    return df


In [73]:
bus_pairs = [
    ("181", "277"), # This corridor through eastern Guayas was incomplete
    ("280", "91"), # Added ring for Quito/Pichincha
    ("60", "102"), # This corridor exists for the 69kV line but goes missing when filtering
    ] 
FIX_LINES_PATH = join(dirs["data/processed/networks"], "line_fix.csv")
df_fix = write_csv_lines(n_exp_clean, bus_pairs, out_file=FIX_LINES_PATH,
                         id_start=801)
print(df_fix)

Wrote 3 lines to c:\Repositories\Repos\pypsa-earth-project\EcuadorElectricGrid\data\processed\networks\line_fix.csv
          Line   ID  num_parallel  v_nom  bus0  bus1     length     dc  \
0  fix_181_277  801           2.0  138.0   181   277   6.026806  False   
1   fix_280_91  802           2.0  138.0   280    91   9.669560  False   
2   fix_60_102  803           2.0  138.0    60   102  31.397940  False   

                                            geometry bounds  ... v_ang_max  \
0  MULTILINESTRING ((-79.5076 -1.9661, -79.5074 -...         ...       inf   
1  MULTILINESTRING ((-78.4113 -0.3306, -78.3299 -...         ...       inf   
2  MULTILINESTRING ((-79.8341 -2.1691, -79.5516 -...         ...       inf   

  sub_network  x_pu  r_pu  g_pu  b_pu  x_pu_eff  r_pu_eff  s_nom_opt  \
0               0.0   0.0   0.0   0.0       0.0       0.0        0.0   
1               0.0   0.0   0.0   0.0       0.0       0.0        0.0   
2               0.0   0.0   0.0   0.0       0.0       0.0 

In [74]:
# 2) Lines expansion
line_df = pd.read_csv(FIX_LINES_PATH)
if "Line" not in line_df.columns:
    raise ValueError("EC_lines_expansion.csv must contain a 'Line' column.")
line_df = line_df.set_index("ID")
line_df = _filter_to_allowed_attrs(n_exp_clean, "Line", line_df)

existing_lines = _update_existing(n_exp_clean, "Line", line_df)
to_add_lines = line_df.drop(index=existing_lines, errors="ignore")
_madd_new(n_exp_clean, "Line", to_add_lines)

In [75]:
networks_dict["network_expanded_no_orphans"] =n_exp_clean

Now, we need three network objects
1) One containing only the original network structure, that is, no buses, trafos, or lines that represent the expansion
2) One that contians the network structure as per the master plan
3) One that contians the nuclear buses 

The final network is the one already containing the nuclear option, so we remove from there




In [76]:
network_nuclear = copy.deepcopy(n_exp_clean)

# Run sanity cheack
def check_network(network):
    _, issues_n = evaluate_network(
        network,
        data_dir = dirs["data/raw/networks"],
        downstream_path = (500,48))
    for k,v in issues_n:
        if "orphan" in k:
            display(v)
            raise ValueError("Network still has orphan buses")
    
    logging.info("Network has no orphan buses")

check_network(network_nuclear)

networks_dict["network_nuclear"]= network_nuclear

Remove the nuclear buses and related components to get only the "productive mix" network

In [77]:
import copy
import logging
import pandas as pd
from os.path import join

def remove_nuclear_assets(network_nuclear, dirs, buses_csv="EC_buses_expansion.csv"):
    """
    Deep-copy the network, then remove:
    - all buses whose Name contains 'nuclear' (case-insensitive) from the expansion buses CSV
    - all lines/transformers connected to those buses

    Logging:
    - Logs the nuclear bus IDs found
    - Logs the line and transformer IDs that are removed

    Returns
    - network_prod_mix: cleaned network (deep copy of input)
    """
    network_prod_mix = copy.deepcopy(network_nuclear)

    # 1) Nuclear bus IDs (robust to NaNs/whitespace; read from expansion CSV)
    bus_csv_path = join(dirs["data/raw/networks"], buses_csv)
    bus_new = pd.read_csv(bus_csv_path)
    mask_nuc = (
        bus_new["Name"].astype(str).str.strip().str.lower().str.contains("nuclear", na=False)
    )
    ids_bus_nuclear = (
        pd.to_numeric(bus_new.loc[mask_nuc, "Bus"], errors="coerce")
        .dropna()
        .astype("Int64")
    )
    logging.info(f"nuclear buses: {ids_bus_nuclear.dropna().tolist()}")

    # 2) Work on full line/trafo tables from the deep-copied network
    lines_existing = network_prod_mix.lines.copy()
    trafo_existing = network_prod_mix.transformers.copy()

    # 3) Align dtypes for membership tests using auxiliary numeric columns
    def _add_numeric_bus_cols(df):
        if df is None or getattr(df, "empty", True):
            return df
        for col in ("bus0", "bus1"):
            if col in df.columns:
                df[col + "_num"] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
        return df

    lines_existing = _add_numeric_bus_cols(lines_existing)
    trafo_existing = _add_numeric_bus_cols(trafo_existing)

    # 4) Filter lines and transformers touching nuclear buses
    ids_lines_nuclear = []
    if lines_existing is not None and not lines_existing.empty:
        mask_lines = (
            lines_existing.get("bus0_num").isin(ids_bus_nuclear)
            | lines_existing.get("bus1_num").isin(ids_bus_nuclear)
        )
        ids_lines_nuclear = lines_existing.index[mask_lines.fillna(False)].tolist()

    ids_trafo_nuclear = []
    if trafo_existing is not None and not trafo_existing.empty:
        mask_trafos = (
            trafo_existing.get("bus0_num").isin(ids_bus_nuclear)
            | trafo_existing.get("bus1_num").isin(ids_bus_nuclear)
        )
        ids_trafo_nuclear = trafo_existing.index[mask_trafos.fillna(False)].tolist()

    logging.info(f"lines hitting nuclear buses: {ids_lines_nuclear}")
    logging.info(f"trafos hitting nuclear buses: {ids_trafo_nuclear}")

    # 5) Remove lines and transformers first, then buses (string IDs for safety)
    i_l, i_t, i_b = 0,0,0
    for l in ids_lines_nuclear:
        network_prod_mix.remove("Line", str(l))
        i_l+=1
    for t in ids_trafo_nuclear:
        network_prod_mix.remove("Transformer", str(t))
        i_t+=1
    for b in ids_bus_nuclear.dropna().tolist():
        network_prod_mix.remove("Bus", str(int(b)))
        i_b+=1

    logging.info(f"{i_l} Lines removes")
    logging.info(f"{i_t} Transformers removed")
    logging.info(f"{i_b} Buses removed")
    logging.info("Finished removing nuclear assets.")
    return network_prod_mix

network_prod_mix = remove_nuclear_assets(network_nuclear, dirs)
networks_dict["network_prod_mix"]= network_prod_mix


Now we remove the expansion assets to reflect the current status

In [78]:
def remove_expansion_assets(
    network_in,
    dirs,
    *,
    buses_csv="EC_buses_expansion.csv",
    lines_csv="EC_lines_expansion.csv",
    trafos_csv="EC_trafo_expansion.csv",
    bus_id_col="Bus",
    line_id_col="ID",
    trafo_id_col="ID",
    ):
    """
    Deep-copy the network, then remove expansion assets:
    - Buses listed in buses_csv (column bus_id_col)
    - Lines listed in lines_csv (column line_id_col)
    - Transformers listed in trafos_csv (column trafo_id_col)

    Logging:
    - Logs IDs found in each CSV
    - Logs how many of each component were removed

    Returns
    - network_base: cleaned network (deep copy of input)
    """
    network_base = copy.deepcopy(network_in)

    base_dir = dirs["data/raw/networks"]
    bus_path = join(base_dir, buses_csv)
    line_path = join(base_dir, lines_csv)
    trafo_path = join(base_dir, trafos_csv)

    # Read CSVs
    bus_df = pd.read_csv(bus_path)
    line_df = pd.read_csv(line_path)
    trafo_df = pd.read_csv(trafo_path)

    # Extract IDs, coerce as needed
    bus_ids = pd.Series([], dtype=object)
    if bus_id_col in bus_df.columns:
        # buses often numeric in expansion files; keep robust casting
        bus_ids = pd.to_numeric(bus_df[bus_id_col], errors="coerce").dropna().astype("Int64")

    line_ids = pd.Series([], dtype=object)
    if line_id_col in line_df.columns:
        line_ids = line_df[line_id_col].dropna().astype(str)

    trafo_ids = pd.Series([], dtype=object)
    if trafo_id_col in trafo_df.columns:
        trafo_ids = trafo_df[trafo_id_col].dropna().astype(str)

    logging.info(f"expansion buses to remove (count={len(bus_ids)}): {bus_ids.dropna().tolist()}")
    logging.info(f"expansion lines to remove (count={len(line_ids)}): {line_ids.tolist()}")
    logging.info(f"expansion transformers to remove (count={len(trafo_ids)}): {trafo_ids.tolist()}")

    # Remove lines and transformers first, then buses
    removed_lines = 0
    for l in line_ids.tolist():
        try:
            network_base.remove("Line", str(l))
            removed_lines += 1
        except Exception:
            pass

    removed_trafos = 0
    for t in trafo_ids.tolist():
        try:
            network_base.remove("Transformer", str(t))
            removed_trafos += 1
        except Exception:
            pass

    removed_buses = 0
    for b in bus_ids.dropna().tolist():
        try:
            network_base.remove("Bus", str(int(b)))
            removed_buses += 1
        except Exception:
            # fall back to raw string if needed
            try:
                network_base.remove("Bus", str(b))
                removed_buses += 1
            except Exception:
                pass

    logging.info(f"{removed_lines} Lines removed")
    logging.info(f"{removed_trafos} Transformers removed")
    logging.info(f"{removed_buses} Buses removed")
    logging.info("Finished removing expansion assets.")

    return network_base

In [79]:
network_base = remove_expansion_assets(network_prod_mix, dirs)

# 2) Lines expansion
line_df = pd.read_csv(FIX_LINES_PATH)
if "Line" not in line_df.columns:
    raise ValueError("EC_lines_expansion.csv must contain a 'Line' column.")
line_df = line_df.set_index("ID")
line_df = _filter_to_allowed_attrs(network_base, "Line", line_df)

existing_lines = _update_existing(network_base, "Line", line_df)
to_add_lines = line_df.drop(index=existing_lines, errors="ignore")
_madd_new(network_base, "Line", to_add_lines)

networks_dict["network_base"]= network_base

In [80]:
network_base.lines[network_base.lines["bus0"]=="280"]

,num_parallel,v_nom,bus0,bus1,length,dc,geometry,bounds,carrier,type,...,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,v_nom_raw
Line,,,,,,,,,,,,,,,,,,,,,
802,2.0,138.0,280,91,9.66956,False,"MULTILINESTRING ((-78.4113 -0.3306, -78.3299 -...",NaN,AC,243-AL1/39-ST1A 20.0,...,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [81]:
#compare the shape of the tee networks for buses, lines and trafos
import pandas as pd

# Define a helper to get basic network element counts
def net_summary(net, name):
    return {
        "Network": name,
        "Buses": len(net.buses),
        "Lines": len(net.lines),
        "Transformers": len(net.transformers),
        "Generators": len(net.generators),
        "Loads": len(net.loads),
        #"Links": len(net.links)
    }

# Build a comparison table
summary = pd.DataFrame([
    net_summary(v, k) for k,v in networks_dict.items()
])

# Display the result
print(summary)


                       Network  Buses  Lines  Transformers  Generators  Loads
0             network_original    313    247            62           0      0
1              network_snapped    313    247            62           0      0
2             network_expanded    336    263            77           0      0
3  network_expanded_no_orphans    319    266            77           0      0
4              network_nuclear    319    266            77           0      0
5             network_prod_mix    317    264            75           0      0
6                 network_base    296    250            62           0      0


In [82]:
#drop them as nc objects so they can be loaded later
# Save networks as NetCDF files (recommended PyPSA format)
import os

output_path = dirs["data/processed/networks"]

# Ensure the directory exists
os.makedirs(output_path, exist_ok=True)

# Save each network to NetCDF in the output folder
for k,v in networks_dict.items():
    filename = join(output_path, f"{k}.nc")
    v.export_to_netcdf(filename)
    logging.info(f"{k} exported to {filename} ")



In [83]:
l = network_base.lines

l[(l.bus0=="104")|(l.bus1=="104")]

,num_parallel,v_nom,bus0,bus1,length,dc,geometry,bounds,carrier,type,...,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,v_nom_raw
Line,,,,,,,,,,,,,,,,,,,,,
72,1.0,69.0,138,104,8.527795,False,MULTILINESTRING ((-79.80759500000018 -2.210634...,MULTIPOINT ((-79.80759500000018 -2.21063489999...,AC,243-AL1/39-ST1A 20.0,...,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.0
73,1.0,69.0,104,59,8.714005,False,MULTILINESTRING ((-79.80759500000018 -2.210634...,"MULTIPOINT ((-79.8351 -2.1701), (-79.807595000...",AC,243-AL1/39-ST1A 20.0,...,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.0
194,1.0,69.0,59,104,9.130528,False,"MULTILINESTRING ((-79.8351 -2.1701, -79.809561...","MULTIPOINT ((-79.8351 -2.1701), (-79.809561100...",AC,243-AL1/39-ST1A 20.0,...,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.0
